In [1]:
def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

<b>Import read and write libraries to file</b>

In [2]:
%%writefile map.wat
(module
    (import "P0lib" "write" (func $write (param i32)))
    (import "P0lib" "read" (func $read (result i32)))
    (memory 1)

Overwriting map.wat


<b>Function used to allocate a new array of a size.</b><br>
<i>Input: 32 bit integer size of array to allocate.</i><br>
<i>Output: 32 bit integer address of array.</i>

In [3]:
%%writefile -a map.wat
    ;; create an array
    (func $arr (param $len i32) (result i32)
        (local $offset i32)                              ;; offset
        (set_local $offset (i32.load (i32.const 0)))     ;; load offset from the first i32

        (i32.store (get_local $offset)                   ;; load the length
                   (get_local $len)
        ) 

        (i32.store (i32.const 0)                         ;; store offset of available space                   
                   (i32.add 
                       (i32.add
                           (get_local $offset)
                           (i32.mul 
                               (get_local $len) 
                               (i32.const 4)
                           )
                       )
                       (i32.const 4)                     ;; the first i32 is the length
                   )
        )
        (get_local $offset)                              ;; (return) the beginning offset of the array.
    )

Appending to map.wat


<b>Function used to get size of array.</b><br>
<i>Input: 32 bit integer address of array.</i><br>
<i>Output: 32 bit integer size of array.</i>

In [4]:
%%writefile -a map.wat
    ;; return the array size
    (func $len_array (param $arr i32) (result i32)
        (i32.load (get_local $arr))
    )

Appending to map.wat


<b>Function used to convert index of array to address offset in array .</b><br>
<i>Input: 32 bit integer address of array.</i><br>
<i>Input: 32 bit integer index of array.</i><br>
<i>Output: 32 bit integer index offset in array.</i>

In [5]:
%%writefile -a map.wat
    ;; convert an element index to the offset of memory
    (func $offset_array (param $arr i32) (param $i i32) (result i32)
        (i32.add
             (i32.add (get_local $arr) (i32.const 4))    ;; The first i32 is the array length 
             (i32.mul (i32.const 4) (get_local $i))      ;; one i32 is 4 bytes
        )
    )

Appending to map.wat


<b>Function used to set index of array to a value.</b><br>
<i>Input: 32 bit integer address of array.</i><br>
<i>Input: 32 bit integer index of array.</i><br>
<i>Input: 32 bit integer value to store at index in array.</i>

In [6]:
%%writefile -a map.wat
    ;; set a value at the index 
    (func $set_array (param $arr i32) (param $i i32) (param $value i32)
        (i32.store 
            (call $offset_array (get_local $arr) (get_local $i)) 
            (get_local $value)
        )  
    )

Appending to map.wat


<b>Function used to get value at index in array.</b><br>
<i>Input: 32 bit integer address of array.</i><br>
<i>Input: 32 bit integer index of array.</i><br>
<i>Output: 32 bit integer value of index in array.</i>

In [7]:
%%writefile -a map.wat
    ;; get a value at the index 
    (func $get_array (param $arr i32) (param $i i32) (result i32)
        (i32.load 
            (call $offset_array (get_local $arr) (get_local $i)) 
        )
    )

Appending to map.wat


<b>Function used to determine mod of 2 integers.</b><br>
<i>Input: 32 bit integer numerator.</i><br>
<i>Input: 32 bit integer denominator.</i><br>
<i>Output: 32 bit integer remainder.</i>

In [8]:
%%writefile -a map.wat
(func $mod (param $x i32) (param $y i32) (result i32)
        (local $q i32)
        (local $r i32)
        i32.const 0
        local.set $q
        local.get $x
        local.set $r
        loop $label0
          local.get $r
          local.get $y
          i32.ge_s
          if
            local.get $r
            local.get $y
            i32.sub
            local.set $r
            local.get $q
            i32.const 1
            i32.add
            local.set $q
            br $label0
          end
        end
        local.get $r
      )

Appending to map.wat


<b>Function used to determine div of 2 integers.</b><br>
<i>Input: 32 bit integer numerator.</i><br>
<i>Input: 32 bit integer denominator.</i><br>
<i>Output: 32 bit integer division.</i>

In [9]:
%%writefile -a map.wat
     (func $div (param $x i32) (param $y i32) (result i32)
        (local $q i32)
        (local $r i32)
        i32.const 0
        local.set $q
        local.get $x
        local.set $r
        loop $label0
          local.get $r
          local.get $y
          i32.ge_s
          if
            local.get $r
            local.get $y
            i32.sub
            local.set $r
            local.get $q
            i32.const 1
            i32.add
            local.set $q
            br $label0
          end
        end
        local.get $q
      )

Appending to map.wat


<b>Function used to determine hash of key for table 1.</b><br>
<i>Input: 32 bit integer length of table 1.</i><br>
<i>Input: 32 bit integer key.</i><br>
<i>Output: 32 bit integer index based on hash value.</i>

In [10]:
%%writefile -a map.wat
    (func $hash1 (param $len i32) (param $key i32) (result i32)
     (call $mod (get_local $key) (get_local $len))
    )

Appending to map.wat


<b>Function used to determine hash of key for table 2.</b><br>
<i>Input: 32 bit integer length of table 2.</i><br>
<i>Input: 32 bit integer key.</i><br>
<i>Output: 32 bit integer index based on hash value.</i>

In [11]:
%%writefile -a map.wat
    (func $hash2 (param $len i32) (param $key i32) (result i32)
     (call $mod (call $div (i32.add (i32.const 4564)(get_local $key))(get_local $len)) (get_local $len))
    )

Appending to map.wat


<b>Function used to allocate a new node for key value pair.</b><br>
<i>Input: 32 bit integer key.</i><br>
<i>Input: 32 bit integer value address.</i><br>
<i>Output: 32 bit integer address of node.</i>

In [12]:
%%writefile -a map.wat
    (func $node (param $key i32) (param $value i32) (result i32)
        (local $offset i32)                              ;; offset
        (set_local $offset (i32.load (i32.const 0)))     ;; load offset from the first i32

        (i32.store (i32.const 0)                         ;; store offset of available space                   
                   (i32.add
                           (get_local $offset)
                           (i32.const 8)
                       )  
        )
        (i32.store (get_local $offset)   ;; store key
                   (get_local $key)
        ) 
     
        (i32.store (i32.add (get_local $offset) (i32.const 4))    ;; store value
                    (get_local $value)
        ) 
        
        (get_local $offset)                              ;; (return) the beginning offset of the node.
    )

Appending to map.wat


<b>Function used to expand map size by 2x.</b><br>
<i>Input: 32 bit integer address of map.</i><br>

In [13]:
%%writefile -a map.wat
    (func $expand_map (param $map i32)
        (local $array1_adr i32)
        (local $array2_adr i32)
        (local $index i32)
        (set_local $array1_adr (i32.load (i32.add (get_local $map) (i32.const 4))))
        (set_local $array2_adr (i32.load (i32.add (get_local $map) (i32.const 8))))
     
        (i32.store (i32.add (get_local $map) (i32.const 4))    
                   (call $arr (i32.mul (i32.load (get_local $array1_adr)) (i32.const 2))))  ;; new array1 of 2x old size
        (i32.store (i32.add (get_local $map) (i32.const 8))    
                   (call $arr (i32.mul (i32.load (get_local $array2_adr)) (i32.const 2))))  ;; new array2 of 2x old size
        (i32.store (get_local $map) (i32.mul (i32.load(get_local $map)) (i32.const 2)))
        (i32.store (i32.add (get_local $map) (i32.const 12)) 
                    (i32.const 0))
        (set_local $index (i32.const 0))
        loop $LOOP1
            (i32.lt_s (get_local $index) (i32.load (get_local $array1_adr)))   
            if
                (if (i32.ne (call $get_array (get_local $array1_adr) (get_local $index)) (i32.const 0)) 
                    (then
                        (call $insert_map (get_local $map) (i32.load (call $get_array (get_local $array1_adr) (get_local $index))) 
                         (i32.load (i32.add (call $get_array (get_local $array1_adr) (get_local $index)) (i32.const 4))))
                    )
                )
                (set_local $index                       
                    (i32.add (get_local $index) (i32.const 1)))
                br $LOOP1
            end
        end
        (set_local $index (i32.const 0))
        loop $LOOP2
            (i32.lt_s (get_local $index) (i32.load (get_local $array2_adr)))   
            if
                (if (i32.ne (call $get_array (get_local $array2_adr) (get_local $index)) (i32.const 0)) 
                    (then
                        (call $insert_map (get_local $map) (i32.load (call $get_array (get_local $array2_adr) (get_local $index))) 
                         (i32.load (i32.add (call $get_array (get_local $array2_adr) (get_local $index)) (i32.const 4))))
                    )
                )
                (set_local $index                       
                    (i32.add (get_local $index) (i32.const 1)))
                br $LOOP2
            end
        end
    )

Appending to map.wat


<b>Function used to allocate a new map.</b><br>
<i>Output: 32 bit integer address of new map.</i>

In [14]:
%%writefile -a map.wat
    (func $map (result i32)
        (local $offset i32)                              ;; offset
        (set_local $offset (i32.load (i32.const 0)))     ;; load offset from the first i32

        (i32.store (get_local $offset)                   ;; initial map size is 8
                   (i32.const 8)
        ) 

        (i32.store (i32.const 0)                         ;; store offset of available space                   
                   (i32.add 
                       (i32.add
                           (get_local $offset)
                           (i32.const 12)
                       )
                       (i32.const 4)                     ;; the first i32 is the size
                   )
        )
        (i32.store (i32.add (get_local $offset) (i32.const 4))    ;; Array1
                   (call $arr (i32.const 4))
        ) 
     
        (i32.store (i32.add (get_local $offset) (i32.const 8))    ;; Array2
                    (call $arr (i32.const 4))
        ) 
        (i32.store (i32.add (get_local $offset) (i32.const 12))    ;; number of initial recrods is 0
                    (i32.const 0)
        )
        
        (get_local $offset)                              ;; (return) the beginning offset of the map.
    )

Appending to map.wat


<b>Function used to check if key in map.</b><br>
<i>Input: 32 bit integer address of map.</i><br>
<i>Input: 32 bit integer key.</i><br>
<i>Output: 32 bit integer <1 if key exists otherwise 0>.</i>

In [15]:
%%writefile -a map.wat
    (func $in_map (param $map i32) (param $key i32) (result i32)
       (local $arr1_index i32)
       (local $arr2_index i32)
       (local $node i32)
       (set_local $arr1_index (call $hash1 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 4)))) (get_local $key)))
       (set_local $arr2_index (call $hash2 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 8)))) (get_local $key)))
       (set_local $node (call $get_array (i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index)))
       (if (i32.and (i32.ne (get_local $node) (i32.const 0))
                    (i32.eq (i32.load (get_local $node))(get_local $key)) ;; check first array
           )
            (then
                (return (i32.const 1))
            )
       )
       (set_local $node (call $get_array (i32.load(i32.add (get_local $map) (i32.const 8))) (get_local $arr2_index)))
       (if (i32.and (i32.ne (get_local $node) (i32.const 0))
                    (i32.eq (i32.load (get_local $node))(get_local $key)) ;; check second array
           )
            (then
                (return (i32.const 1))
            )
       )
       (i32.const 0)
    )

Appending to map.wat


<b>Function used to insert key-value pair in map.</b><br>
<i>Input: 32 bit integer address of map.</i><br>
<i>Input: 32 bit integer key.</i><br>
<i>Input: 32 bit integer value address.</i><br>

In [16]:
%%writefile -a map.wat
    (func $insert_map (param $map i32) (param $key i32) (param $value i32)
       (local $arr1_index i32)
       (local $arr2_index i32)
       (local $map_size i32)
       (local $node i32)
       (local $existing i32)
       (local $table i32)
       (local $max_replace i32)
       (local $temp i32)
       (local $numOfRecords i32)
       (if (i32.eq (call $in_map (get_local $map) (get_local $key)) (i32.const 1))
           (then 
            (call $update_key (get_local $map) (get_local $key) (get_local $value))
           )
        (else 
       (set_local $numOfRecords (i32.load(i32.add (get_local $map) (i32.const 12))))
       (set_local $map_size (call $len_map (get_local $map)))
       (if (i32.ge_s 
                    (get_local $numOfRecords) 
                    (call $div (get_local $map_size) (i32.const 2)) ;; if insert into table 1
                ) 
                    (then
                     (call $expand_map (get_local $map))
                    )
       )
       (set_local $max_replace (i32.const 0))
       (set_local $table (i32.const 2))
       (set_local $node (call $node (get_local $key) (get_local $value)))
       (set_local $arr1_index (call $hash1 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 4)))) (get_local $key)))
       (set_local $arr2_index (call $hash2 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 8)))) (get_local $key)))
       (set_local $existing (call $get_array (i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index))) ;; set existing to node being pushed out

       (call $set_array (
           i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index) (get_local $node)) ;; insert new node into first array
       (i32.store (i32.add (get_local $map) (i32.const 12)) 
                    (i32.add (i32.load(i32.add (get_local $map) (i32.const 12))) (i32.const 1))) ;; numOfRecords + 1
       loop $LOOP
            (i32.and 
             (i32.lt_s (get_local $max_replace) (i32.const 5)) ;; if max replacements is under 5
             (i32.ne (get_local $existing) (i32.const 0)) ;; if exisitng points to a node (not equal to 0)
            )   
            if      
                (if (i32.eq 
                    (get_local $table) 
                    (i32.const 1) ;; if insert into table 1
                ) 
                    (then
                        (set_local $key (i32.load (get_local $existing))) ;; get the key of the existing node
                        (set_local $arr1_index (call $hash1 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 4)))) (get_local $key))) ;; hash key and find index in array 1
                        (set_local $temp (call $get_array (i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index))) ;; set temp to the value in the index
                        (call $set_array (
           i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index) (get_local $existing)) ;; store the exisitng in the index
                        (set_local $existing (get_local $temp)) ;; set exisitng to temp (what was in the index before)
                        (set_local $table (i32.const 2)) ;; set table to 2
                    )
                    (else ;; if insert into table 2
                       (set_local $key (i32.load (get_local $existing)))
                       (set_local $arr2_index (call $hash2 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 8)))) (get_local $key)))
                       (set_local $temp (call $get_array (i32.load(i32.add (get_local $map) (i32.const 8))) (get_local $arr2_index)))
                        (call $set_array (
           i32.load(i32.add (get_local $map) (i32.const 8))) (get_local $arr2_index) (get_local $existing))
                        (set_local $existing (get_local $temp))
                        (set_local $table (i32.const 1)) ;; set table to 1
                    )
                )
                (set_local $max_replace                       
                    (i32.add (get_local $max_replace) (i32.const 1))) ;; max_replace + 1
                br $LOOP
            end
        end
        (if (i32.eq 
                    (get_local $existing) 
                    (i32.const 0) ;; if nothing in index
                ) 
            (then
                (call $set_array (
           i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index) (get_local $node))
            )
        )
        (if (i32.and (i32.ne (get_local $existing) (i32.const 0)) (i32.eq (get_local $max_replace) (i32.const 5)))
            (then
                (call $expand_map (get_local $map))
                (call $insert_map (get_local $map) (i32.load (get_local $existing)) 
                     (i32.load (i32.add (get_local $existing) (i32.const 4))))  
            )
        ) 
        ))
    )

Appending to map.wat


<b>Function used to update value of key in map.</b><br>
<i>Input: 32 bit integer address of map.</i><br>
<i>Input: 32 bit integer key.</i><br>
<i>Input: 32 bit integer value address.</i><br>

In [17]:
%%writefile -a map.wat
    (func $update_key (param $map i32) (param $key i32) (param $value i32)
       (local $arr1_index i32)
       (local $arr2_index i32)
       (local $node i32)
       (set_local $arr1_index (call $hash1 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 4)))) (get_local $key)))
       (set_local $arr2_index (call $hash2 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 8)))) (get_local $key)))
       (set_local $node (call $get_array (i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index)))
       (if (i32.and (i32.ne (get_local $node) (i32.const 0))
                    (i32.eq (i32.load (get_local $node))(get_local $key)) ;; check in Array1
           )
            (then
                (i32.store (i32.add (get_local $node) (i32.const 4)) (get_local $value)) ;; set value address
            )
       )
       (set_local $node (call $get_array (i32.load(i32.add (get_local $map) (i32.const 8))) (get_local $arr2_index)))
       (if (i32.and (i32.ne (get_local $node) (i32.const 0))
                    (i32.eq (i32.load (get_local $node))(get_local $key)) ;; check in Array2
           )
            (then
                (i32.store (i32.add (get_local $node) (i32.const 4)) (get_local $value)) ;; set value address
            )
       )
    )

Appending to map.wat


<b>Function used to get value stored at key in map.</b><br>
<i>Input: 32 bit integer address of map.</i><br>
<i>Input: 32 bit integer key.</i><br>
<i>Output: 32 bit integer value address.</i><br>

In [18]:
%%writefile -a map.wat

    (func $get_map (param $map i32) (param $key i32) (result i32)
       (local $arr1_index i32)
       (local $arr2_index i32)
       (local $node i32)
       (local $map_size i32)
       (set_local $map_size (call $len_map (get_local $map)))
       (set_local $arr1_index (call $hash1 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 4)))) (get_local $key)))
       (set_local $arr2_index (call $hash2 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 8)))) (get_local $key)))

       (set_local $node (call $get_array (i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index)))
       (if (i32.and (i32.ne (get_local $node) (i32.const 0))
                    (i32.eq (i32.load (get_local $node))(get_local $key)) ;; check in Array1
           )
            (then
                (return (i32.load (i32.add(get_local $node)(i32.const 4))))
            )
       )
       (set_local $node (call $get_array (i32.load(i32.add (get_local $map) (i32.const 8))) (get_local $arr2_index)))
       (if (i32.and (i32.ne (get_local $node) (i32.const 0))
                    (i32.eq (i32.load (get_local $node))(get_local $key)) ;; check in Array2
           )
            (then
                (return (i32.load (i32.add(get_local $node)(i32.const 4))))
            )
       )
       (i32.const -1)
    )
 

Appending to map.wat


<b>Function used to get size of map.</b><br>
<i>Input: 32 bit integer address of map.</i><br>
<i>Output: 32 bit integer size of map.</i><br>

In [19]:
%%writefile -a map.wat
     ;; return the array length
    (func $len_map (param $map i32) (result i32)
        (i32.load (get_local $map))
    )


Appending to map.wat


<b>Function used to get number of records in map.</b><br>
<i>Input: 32 bit integer address of map.</i><br>
<i>Output: 32 bit integer number of records in map.</i><br>

In [20]:
%%writefile -a map.wat
    (func $numOfRecords (param $map i32) (result i32)
        (i32.load (i32.add (get_local $map) (i32.const 12)))
    )

Appending to map.wat


<b>Sample tests in Web Assembly</b>

In [21]:
%%writefile -a map.wat
    (func $main
        (local $map1 i32)
        ;; The first i32 records the beginning offset of available space
        ;; so the initial offset should be 4 (bytes)
        (i32.store (i32.const 0) (i32.const 4))     

        (set_local $map1 (call $map))   ;; create a map and assign to $map1
        (call $len_map (get_local $map1))
        call $write                                   ;; print map initial length of 8
        (call $len_array (i32.load(i32.add (get_local $map1) (i32.const 4))))
        call $write
        (call $insert_map (get_local $map1) (i32.const 9) (i32.const 2345)) ;; insert 
        (call $insert_map (get_local $map1) (i32.const 9) (i32.const 2346)) ;; insert 
        (call $insert_map (get_local $map1) (i32.const 1) (i32.const 235)) ;; insert 
        (call $insert_map (get_local $map1) (i32.const 5) (i32.const 236)) ;; insert 
        (call $insert_map (get_local $map1) (i32.const 13) (i32.const 123)) ;; insert 
        (call $insert_map (get_local $map1) (i32.const 12) (i32.const 124)) ;; insert 
        (call $insert_map (get_local $map1) (i32.const 123) (i32.const 235)) ;; insert 
        (call $insert_map (get_local $map1) (i32.const 5423) (i32.const 236)) ;; insert 
        (call $insert_map (get_local $map1) (i32.const 234) (i32.const 123)) ;; insert 
        (call $insert_map (get_local $map1) (i32.const 243) (i32.const 124)) ;; insert 
        (call $get_map (get_local $map1) (i32.const 9)) ;; get key 9
        call $write
        (call $get_map (get_local $map1) (i32.const 1)) ;; get key 2
        call $write
        (call $get_map (get_local $map1) (i32.const 5)) ;; get key 2
        call $write
        (call $get_map (get_local $map1) (i32.const 13)) ;; get key 2
        call $write
        (call $get_map (get_local $map1) (i32.const 12)) ;; get key 2
        call $write
        (call $numOfRecords (get_local $map1)) ;; get number of records
        call $write
        (call $write (call $len_map (get_local $map1))) ;; new size of map
        (call $write (call $in_map (get_local $map1) (i32.const 10))) ;; is key = 10 in map (false = 0)
    )
     (func $testingMap
        ;;declaring map 
        (local $myMap i32) (i32.store (i32.const 0) (i32.const 4))
        (set_local $myMap (call $map))
        ;;getting length
        get_local $myMap
        call $len_map
        call $write      
        ;; adding <1, 19>
        get_local $myMap
        i32.const 1 
        i32.const 19
        call $insert_map
        ;; fetching myMap[1]
        get_local $myMap
        i32.const 1 
        call $get_map
        call $write
        ;; fetching len(myMap)
        get_local $myMap
        call  $numOfRecords
        call $write
        ;;mass insertion
        (call $insert_map (get_local $myMap) (i32.const 2) (i32.const 121))
        (call $insert_map (get_local $myMap) (i32.const 4) (i32.const 343))
        (call $insert_map (get_local $myMap) (i32.const 11) (i32.const 16))
        (call $insert_map (get_local $myMap) (i32.const 22) (i32.const 45))
        ;;overwriting <1,19> to <1,2>
        (call $insert_map (get_local $myMap) (i32.const 1) (i32.const 2))
        get_local $myMap
        i32.const 1 
        call $get_map
        call $write
        (call $numOfRecords (get_local $myMap))
        call $write
      ;; output should be, size of map, fetching <1,19>, num of records, fetching <1,2>, num records again
    )
    ;; (start $testingMap)
    
    (start $main)
)

Appending to map.wat


In [22]:
!wat2wasm map.wat
runpywasm("map.wasm")

8
4
2346
235
236
123
124
9
32
0
